## Exercise Sheet 8
### Analyzing Sentence Structure

**Exercise 1**


Write a recursive function to traverse a tree and return the depth of the tree, such that a tree with a single node would have depth zero. (Hint: the depth of a subtree is the maximum depth of its children, plus one.) Test your function with the two trees produced by the `ChartParser` for the`groucho_grammar` and the sentence “I shot an elephant in my pajamas” from this chapter. The result can be verified with the `Tree.height()` function.

In [144]:
import nltk
import re

def get_tree_depth_recursively(tree_string: str, current_depth: int = 1):  
    idx = 0
    tree_depth = current_depth
    
    # Outer loop: Identify subtrees on same level.
    while idx < len(tree_string):
        bracket_balance = 0
        subtree_start_idx = idx
        subtree_end_idx = 0
        
        # Inner loop: Iterate as long as current tree is not closed.
        for char in tree_string[idx:]:
            if char == "(":
                bracket_balance += 1
            elif char == ")":
                bracket_balance -= 1    
                subtree_end_idx = idx
            
            # If closing bracket was found: Do the same for this subtree.
            if bracket_balance == 0:
                max_subtree_depth = get_tree_depth_recursive(
                    tree_string[subtree_start_idx + 1:subtree_end_idx],
                    current_depth + 1
                )
                tree_depth = max_subtree_depth if max_subtree_depth > tree_depth else tree_depth
                break
                
            # Keep track of character index in stringified tree.
            idx += 1

    return tree_depth

def get_tree_depth_sequentially(tree_string: str):
    tree_depth = 0
    current_tree_depth = 1
    
    # Iterate over string representation of tree, count occurences of "(" before brackets are closed again.
    for char in tree_string:
        if char == "(":
            current_tree_depth += 1
        elif char == ")":
            current_tree_depth -= 1

        tree_depth = current_tree_depth if current_tree_depth > tree_depth else tree_depth
        
    return tree_depth
    
groucho_grammar = nltk.CFG.fromstring("""
    S -> NP VP
    PP -> P NP
    NP -> Det N | Det N PP | 'I'
    VP -> V NP | VP PP
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
    """)

sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)
for tree in parser.parse(sent):
    print("Original:\n" + str(tree))
    tree_preproc = re.sub(r'[^())]+', '', str(tree))
    print("\nPreprocessed:\n" + tree_preproc)
    rec_tree_depth = get_tree_depth_recursively(tree_preproc)
    seq_tree_depth = get_tree_depth_sequentially(str(tree))
        
    if rec_tree_depth == tree.height() and seq_tree_depth == tree.height():
        print("\nTree depth: " + str(rec_tree_depth))
    else:
        print("Error - tree depth mismatch.")
    print("\n-------------------\n")

Original:
(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))

Preprocessed:
(()((()(()()))(()(()()))))

Tree depth: 6

-------------------

Original:
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))

Preprocessed:
(()(()(()()(()(()())))))

Tree depth: 7

-------------------



**Exercise 3**

Write a recursive function `bracketing(tree)` that produces a nested bracketing for a `tree`, leaving out the leaf nodes, and displaying the non-terminal labels after their sub-trees. Consecutive categories should be separated by space. Test your function with the tree:

In [159]:
from nltk.corpus import treebank
import re

def bracketing(tree: str, depth: int = 0):  
    idx = tree.find("(")
    tree_bracketing = ""
    non_terminals = ""
    
    # Outer loop: Identify subtrees on same level.
    while idx != -1 and idx < len(tree):
        bracket_balance = 0        
        subtree_start_idx = idx + 1
        subtree_end_idx = 0
        
        # Inner loop: Iterate as long as current tree is not closed.
        for char in tree[idx:]:
            if char == "(":
                bracket_balance += 1
            elif char == ")":
                bracket_balance -= 1    
                subtree_end_idx = idx
            
            # If closing bracket was found: Do the same for this subtree.
            if bracket_balance == 0:
                # Extract non-terminals.
                non_terminals = tree[0:tree.find("(")]
                # Call recursion.
                tree_bracketing += bracketing(tree[subtree_start_idx:subtree_end_idx], depth + 1)
                
                # Exit (inner) loop.
                break
                
            # Keep track of character index in stringified tree.
            idx += 1

    # If subtree(s) exists: Add brackets around content, use and clean content before first bracket as non-terminals.
    if idx != -1:
        # Avoid unnecessary brackets. Use rstrip() to eliminate superfluous whitespaces before closing bracket.
        tree_bracketing = "[" + tree_bracketing.rstrip() + "]" if len(non_terminals) > 0 else tree_bracketing
        
    # If no subtrees exist, but the current subtree has content/nonterminals: 
    # Use and clean all content before first bracket as non-terminal.
    elif tree:
        non_terminals = tree.split()[0].strip()
        
    # Add non-terminals at end.
    if tree_bracketing:
        return tree_bracketing + non_terminals    
    else:
        return non_terminals + " " if non_terminals else non_terminals

t = treebank.parsed_sents("wsj_0001.mrg")[0]
print("Original: \n" + str(t))
# Start bracketing with cleaned input.
print("\nPreprocessed: \n" + re.sub(r'[\n]+', '', ' '.join(str(t).split())))
# Apply recursive bracketing.
print("\nBracketing:\n" + bracketing(
    re.sub(r'[\n]+', '', ' '.join(str(t).split()))
))
print("Reference solution:\n[[[NNP NNP]NP , [[CD NNS]NP JJ]ADJP ,]NP-SBJ [MD [VB [DT NN]NP [IN [DT JJ NN]NP]PP-CLR [NNP CD]NP-TMP]VP]VP .]S")

Original: 
(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))

Preprocessed: 
(S (NP-SBJ (NP (NNP Pierre) (NNP Vinken)) (, ,) (ADJP (NP (CD 61) (NNS years)) (JJ old)) (, ,)) (VP (MD will) (VP (VB join) (NP (DT the) (NN board)) (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director))) (NP-TMP (NNP Nov.) (CD 29)))) (. .))

Bracketing:
[[[NNP NNP]NP , [[CD NNS]NP JJ]ADJP ,]NP-SBJ [MD [VB [DT NN]NP [IN [DT JJ NN]NP]PP-CLR [NNP CD]NP-TMP]VP]VP .]S 
Reference solution:
[[[NNP NNP]NP , [[CD NNS]NP JJ]ADJP ,]NP-SBJ [MD [VB [DT NN]NP [IN [DT JJ NN]NP]PP-CLR [NNP CD]NP-TMP]VP]VP .]S
